In [ ]:
import warnings; 
warnings.simplefilter('ignore')
# birkav gereksiz hatayi gormezden gelebiliriz

# 0. Install and Import Dependencies

In [ ]:
!pip install pystan==2.19.1.1
!pip install fbprophet
# pystan ve fbprophet indirme

In [ ]:
import pandas as pd
from fbprophet import Prophet

# 1. Read in Data and Process Dates

In [ ]:
df = pd.read_csv("../input/timeseriesforecasting/forecast.csv")

In [ ]:
df.head(10)
# ilk 10 veriye bakacak olursak Store ve Product ozellikleri
# her veri icin ayni gozukuyor. Oyle mi degil mi diye bir bakalim

In [ ]:
print(df['Product'].unique())
print(df['Store'].unique())
# Kodu calistirinca unique yani hepsinde ayni oldugunu anliyoruz

In [ ]:
df.describe()
# 3 sayili (float,int vs.) deger goruyoruz. 
# Store degerleri ise veri tipi sayili olmayan

In [ ]:
df.dtypes
# Veri tiplerine bakalim ; 

In [ ]:
df['Year'] = df['Time Date'].apply(lambda x: str(x)[-4:])
df['Month'] = df['Time Date'].apply(lambda x: str(x)[-6:-4])
df['Day'] = df['Time Date'].apply(lambda x: str(x)[:-6])
df['ds'] = pd.DatetimeIndex(df['Year']+'-'+df['Month']+'-'+df['Day'])
# integer olarak verilmis Time Date degerlerini daha anlamli hale getiriyoruz
# Parcalara ayirarak bunlari daha kullanilabilir yapiyoruz. Bunu da ;
# appyle icine lambda kullanarak yaptik. 

In [ ]:
df.head()
# int olan degiskenimizin parcali halini goruyoruz. Ayrica
# 'ds' adinda datetime degiskenimizi de uzun hali olarak goruyoruz

In [ ]:
df.dtypes
# goruldugu gibi veri tiplerinin son hali

In [ ]:
df = df.loc[(df['Product']==2667437) & (df['Store']=='QLD_CW_ST0203')]
df.drop(['Time Date', 'Product', 'Store', 'Year', 'Month', 'Day'], axis=1, inplace=True)
df.columns = ['y', 'ds']
# Boylece isimize yaramayan kolonlari drop ettik ve iki kolon uzerine yogunlastik
# axis = 0 satirlarla, axis = 1 sutunlarla islem yapar.
# inplace = True sayesinde islemi serinin kendisi uzerinde yapiyoruz.

In [ ]:
df.head()
# kullanacagimiz 2 kolonumuz ve verilerimiz hazir.

# 2. Train Model

In [ ]:
    m = Prophet(interval_width=0.95, daily_seasonality=True) # import ederek aldigimiz prophet
    # guven araligimiz 0.95 olarak belirledik. 
    # tarihlerimiz gun gun ilerledigi icin de daily_seasonality true olarak belirledik, belirtmeye cok da gerek yok
    model = m.fit(df) # egitimimiz icin fit methodunu cagirdik. Bu sayede verilerimizi egitebilecegiz.
    # Fit yontemi cagirilinca, veri noktalari icin en iyi temsili islevi tahmin eder
    
    # Ve su an egitilmis bir modelimiz elimizde

Egitilmis modelimiz ile bazi tahminlerde bulunalim...

# 3. Forecast Away

In [ ]:
future = m.make_future_dataframe(periods=100,freq='D') # takvimde gun sikligi D ile gosteriliyor
forecast = m.predict(future) # fit ve predict metotlari diger metotlardan bildigimiz metotlar
forecast.head()
# en sonda tahminimizi temsil eden ythat ve diger tum verilerimiz de burada
# ileriye donuk tahminlerde bulunduk ve ne kadar gun ileriye gitsek de hala bulmaya devam edecektir.
# ayrica daily seasonality bilgilerimizi de gorebiliyoruz. 

In [ ]:
forecast.tail()
# head in tersi son 5 satiri bize gosterir

In [ ]:
# sadece tarihlerimizi ve gercek tahminimizi cikarmak istiyorsak,
# hizli ve basit bir filtreleme yontemi yapabiliriz.
forecast[['ds', 'yhat']]
# sadece tarih ve ileriki tahminlerimizi gosterdik. İlk kisim
# gecmis veriler ve son kisim bizim tahminlerimiz. 

In [ ]:
plot1 = m.plot(forecast)
# mavi cizgi tahminimizi temsil eder. siyah noktalar ise tahmin edilen verilerimiz

In [ ]:
plt2 = m.plot_components(forecast)
# verileri daha ayrintili olarak gorsele dokelim ve gunlerin tahminlere nasil bir etkisi olduguna bakalim;